#  Use effective heuristics for alpha sizing

In [1]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline 

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append('../')

## Background

$$ 
\text{PnL} = \text{idiosyncratic PnL} + \text{factor PnL} 
$$

$$ 
\text{factor PnL} = \sum_{k=1}^K \times \text{factor PnL}_k
$$

$$ 
 \text{factor PnL}_k = \text{exposure}_k \times \text{factor return}_k 
$$

$$ 
\text{exposure}_k =  \sum_{i=1}^N\beta_{ik}\times \text{NMV}_i
$$


## Tactical 

**Procedure 7.1: Tactical Portfolio Construction**

1. **Start with a set of fundamental theses (Section 6.2).**
   - Develop and clearly define your investment theses based on fundamental analysis.

2. **Convert these theses into a dollar position using Procedure 6.2.**
   - Apply Procedure 6.2 to translate your fundamental convictions into specific dollar positions in the portfolio.

3. **Perform a risk decomposition.**
   - Analyze the portfolio to break down the sources of risk, identifying the contributions from various factors and idiosyncratic components.

4. **If the percentage of idiosyncratic variance is low, identify the group of factors responsible for it.**
   - For guidance regarding acceptable levels of factor risk, see Section 7.2.1.
   - Assess the portfolio's factor exposures and determine if the idiosyncratic risk is sufficiently low. If not, find out which factors contribute to the risk.

5. **Identify the largest factor or factors responsible for the low percentage of idiosyncratic variance.**
   - Include factors that have exposures exceeding the single-factor exposures, as discussed in Section 7.2.4.
   - Pinpoint the key factors driving the portfolio's risk and those with significant exposures.

6. **Identify the trades by asset that both reduce overall factor risk and the specific factor exposure.**
   - Choose trades compatibly with your fundamental convictions and with your goals to reduce or increase the Gross Market Value (GMV) of the portfolio.
   - Determine asset-specific trades that align with your investment theses while managing factor risks effectively.

7. **Change the positions in a direction that is:**
   - (a) Compatible with your convictions;
   - (b) Reducing factor risk to an acceptable level;
   - (c) Reducing exposures to individual factors that you are currently targeting.
   - Adjust the portfolio to align with your fundamental beliefs, ensuring factor risks are mitigated to acceptable levels, and focus on targeted factor exposures.

By following these steps, you can tactically construct a portfolio that balances your fundamental investment convictions with disciplined risk management.

**11.7 Tactical Portfolio Optimization**

Given the definitions and parameters:

- **w**: starting portfolio.
- **x**: traded portfolio.

### Parameters
- **C**: Matrix of custom factor loadings (e.g., short interest, hedge fund ownership data, ESG loadings, yield sensitivity).
- **TC**: Market impact model, defined as \( TC(x) = \kappa \sum \sigma_i \beta \left( \frac{|x_i|}{v_i} \right)^{\gamma} \) where:
  - \( v_i \): Average daily trading volume (in dollars).
  - \( \sigma_i \): Volatility.
  - \( \beta \approx 1 \)
  - \( \gamma \approx \frac{3}{2} \)
  - \( \kappa > 0 \): Region-calibrated constant.
- **lb, ub**: Lower and upper bounds on factor exposures (usually \( lb = -ub \)).
- **lc, uc**: Lower and upper bounds on custom exposures (usually \( lc = -uc \)).
- **d**: Vector of idiosyncratic volatility.
- **\(\sigma_f, \sigma_t\)**: Upper bounds for factor risk and total risk, respectively.
- **g**: Lower bound on "edge" or Gross Market Value (GMV).
- **E**: Maximum tracking error between initial and final portfolio.

### Optimization Problem
We need to solve the following optimization problem:

#### Objective
Minimize \( TC(x) \)

#### Subject to:
1. **Final Portfolio**: \( v = w + x \)
2. **Maximum Tracking Error**: \( ||x|| \leq E \)
3. **Ancillary Variable for GMV**: 
   - \( w + x \geq y \)
   - \(- (w + x) \geq z \)
4. **Final Exposures**: \( b = B'v \)
5. **Final Custom Exposures**: \( c = C'v \)
6. **Final Factor Risk Bound**: \( b' \Omega_f b \leq \sigma_f^2 \)
7. **Final Exposure Bounds**: \( lb \leq b \leq ub \)
8. **Final Custom Exposure Bounds**: \( lc \leq c \leq uc \)
9. **Minimum GMV**: \( \sum (y_i + z_i) \geq g \)
10. **Non-negativity Constraints**: \( y \geq 0 \), \( z \geq 0 \)

### Variants
Allow the user to select the objective function from two options:
1. **Distance from Starting Portfolio**: \( x'x \)
2. **Idiosyncratic Tracking Error**: \( x'\Omega_\epsilon x \)

### Implementation

Here is a possible Python implementation using a simplified approach with placeholders for the actual optimization routines, which would typically require a numerical optimization library such as SciPy or CVXPY:

```python
import numpy as np
import cvxpy as cp

# Define the parameters
w = np.array([1, 2, 3])  # Example starting portfolio
C = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])  # Custom factor loadings
B = np.array([[0.5, 0.5, 0], [0.3, 0.3, 0.4], [0.2, 0.4, 0.4]])  # Example factor loadings
Omega_f = np.diag([0.1, 0.2, 0.3])  # Factor covariance matrix
d = np.array([0.1, 0.2, 0.3])  # Idiosyncratic volatility
v = np.array([1000, 1500, 2000])  # Average daily trading volume
sigma_i = np.array([0.05, 0.07, 0.06])  # Volatility
beta = 1
gamma = 1.5
kappa = 0.1
lb = np.array([-0.5, -0.5, -0.5])  # Lower bounds on factor exposures
ub = np.array([0.5, 0.5, 0.5])  # Upper bounds on factor exposures
lc = np.array([-0.3, -0.3, -0.3])  # Lower bounds on custom exposures
uc = np.array([0.3, 0.3, 0.3])  # Upper bounds on custom exposures
sigma_f = 0.15  # Upper bound for factor risk
sigma_t = 0.20  # Upper bound for total risk
g = 0.05  # Lower bound on edge or GMV
E = 0.1  # Maximum tracking error

# Define the optimization variables
x = cp.Variable(len(w))
y = cp.Variable(len(w), nonneg=True)
z = cp.Variable(len(w), nonneg=True)
v_final = w + x
b = B.T @ v_final
c = C.T @ v_final

# Define the objective function for market impact model
TC = kappa * cp.sum(sigma_i * beta * cp.power(cp.abs(x) / v, gamma))

# Define the constraints
constraints = [
    cp.norm(x) <= E,
    w + x >= y,
    -(w + x) >= z,
    b @ Omega_f @ b <= sigma_f**2,
    lb <= b,
    b <= ub,
    lc <= c,
    c <= uc,
    cp.sum(y + z) >= g
]

# Define the problem and solve it
objective = cp.Minimize(TC)
problem = cp.Problem(objective, constraints)
problem.solve()

# Print the results
print("Optimal traded portfolio x:")
print(x.value)
print("Final portfolio v:")
print(v_final.value)
```

This implementation provides a basic structure for the tactical portfolio optimization problem. It sets up the optimization variables, defines the objective function, and specifies the constraints. The actual numerical solution is obtained using the CVXPY library, which is well-suited for convex optimization problems. Adjustments may be needed based on specific details and data relevant to your problem context.

## Strategic

## Systematic Hedging and Portfolio Management

### Three Approaches to Simplify Portfolio Management

1. **Automatic Hedging Program**:
   - Regularly run an optimization to minimize transaction costs while keeping portfolio factor exposure, factor risk, and single-factor exposures within limits. This involves trading a broad, liquid universe of stocks without altering the original stock sizes chosen by the portfolio manager.

2. **Factor-Mimicking Portfolios**:
   - Create synthetic assets that mimic factor returns, allowing the portfolio manager to reduce factor risk by trading these assets instead of individual stocks. This approach maintains the core positions of the portfolio unaffected.

3. **Fully Automated Portfolio Construction**:
   - Input investment ideas into a portfolio optimizer, which generates and updates the portfolio to reflect core ideas efficiently, maintain factor risk within limits, and adapt to market conditions and new ideas systematically.

## Takeaway Messages

1. **Generate Convictions**:
   - Use fundamental research to form your investment convictions.

2. **Turn Convictions into Positions**:
   - Use a proportional volatility scaling rule to convert your convictions into positions.

3. **Measure and Update Portfolio Risk**:
   - Perform risk decomposition and update your portfolio considering factor risk.

4. **Levels of Portfolio Management**:
   - Marginal risk-reducing trades
   - Single-portfolio optimization
   - Systematic factor hedging
   - Trading factor portfolios
   - Fully automated portfolio management

5. **Steps for Marginal Risk-Reducing Trades**:
   - Add/update an idea in the portfolio pro-forma.
   - Check the change in % idiosyncratic variance (%idio var).
   - Identify the sources of factor risk if there is a sizable change.
   - Identify stocks with the largest marginal risk contributions and update their positions accordingly.

6. **Risk Limits**:
   - Set risk limits on %idio, style, and factor variance compatible with your investment style.
   - Avoid going below 70% idiosyncratic variance, and almost never below 50%.
   - Maintain a single-position GMV limit, adjusting it based on your conviction and portfolio size.
   - Keep a single-factor exposure limit based on your loss tolerance and worst-case factor returns.

7. **Automated Hedging**:
   - For higher productivity, consider implementing an automated hedging program to generate perfect hedges for your portfolio.